In [28]:
import numpy as np

In [6]:
#daily daily data into weekly data
#AAPL
import pandas as pd


yahoo = pd.read_csv("Finance_Data_Daily_Limited.csv")

In [7]:
yahoo.dtypes

open        float64
high        float64
low         float64
close       float64
adjclose    float64
volume        int64
ticker       object
Date         object
dtype: object

In [10]:
#first, we want to take the limited data and create a subset with only apple tickers
yahoo_apple = yahoo[yahoo['ticker'] == 'AAPL']

In [23]:
#now, we want a function that condenses the rows into weekly rows of data
#yahoo_apple['Date'] = pd.to_datetime(yahoo_apple['Date'])

weekly_yahoo_apple = yahoo_apple.groupby(pd.Grouper(key='Date', freq='W-MON',closed='left')).agg(
    start_date=('Date', 'min'),
    end_date=('Date', 'max'),
    open=('open', 'first'),
    high=('high', 'max'),
    low=('low', 'min'),
    close=('close', 'last'),
    volume=('volume', 'mean')
).reset_index()

# Display the first few rows
print(weekly_yahoo_apple.head(3))



    

        Date start_date   end_date        open        high         low  \
0 2022-01-10 2022-01-03 2022-01-07  177.830002  182.940002  171.029999   
1 2022-01-17 2022-01-10 2022-01-14  169.080002  177.179993  168.169998   
2 2022-01-24 2022-01-18 2022-01-21  171.509995  172.539993  162.300003   

        close       volume  
0  172.169998   96389800.0  
1  173.070007   84531140.0  
2  162.410004  100010275.0  


In [25]:
weekly_yahoo_apple.shape

(104, 8)

In [26]:
#now that the yahoo data has been changed to display weekly info, we can join it with the perigon data for AAPL. 
perigon_apple = pd.read_csv("dataAAPL.csv")

In [38]:
#here are the cleaning functions for the perigon data
#different cleaning functions to apply to attributes
#only take company values when they have corresponding symbols in the data frame


def clean_companies_dictionary(example):
        if example['symbols'] != '':
            return [{'name': example['name'], 'symbols': example['symbols']}]

def clean_source_dictionary(example):
    return {'domain': example['domain']}

#the below function will only return dictionaries where the value of the 'type' attribute is 'ORG'.
#def clean_entities_dictionary(example):
#    try:
#        return {example['data']: example['mentions']}
#    except example['type'] != 'ORG':
 #       return None
        
def clean_sentiment_positive_dictionary(example):
    return example['positive']
        
def clean_sentiment_negative_dictionary(example):
    return example['negative']

def clean_sentiment_neutral_dictionary(example):
    return example['neutral']

In [39]:
#let's run the cleaning operations for the perigon Apple data

#adding a time attribute based on splitting after the T. 

perigon_apple['pubTime'] = perigon_apple['pubDate'].apply(lambda x: x.split('T')[1].split('+')[0])
perigon_apple['addTime'] = perigon_apple['addDate'].apply(lambda x: x.split('T')[1].split('+')[0])

#cleaning the date formats using string splitting
perigon_apple['pubDate_Clean'] = perigon_apple['pubDate'].apply(lambda x: x.split('T')[0])
perigon_apple['addDate_Clean'] = perigon_apple['addDate'].apply(lambda x: x.split('T')[0])

#creating three new attributes for the positive, negative, and neutral scores
#each score is the percentage of the article that is either positive, negative, or neutral. 
#these three attributes for any given example add up to 1. 
perigon_apple['positive_sentiment'] = perigon_apple['sentiment'].apply(clean_sentiment_positive_dictionary)
perigon_apple['negative_sentiment'] = perigon_apple['sentiment'].apply(clean_sentiment_negative_dictionary)
perigon_apple['neutral_sentiment'] = perigon_apple['sentiment'].apply(clean_sentiment_neutral_dictionary)

perigon_apple['n_companies'] = perigon_apple['companies'].apply(len)

perigon_apple['source'] = perigon_apple['source'].apply(clean_source_dictionary)

#We still need to figure out how to implement the 'mentions function'



TypeError: string indices must be integers, not 'str'

In [42]:
##changing the dates when an article is late in the day

def adjust_date_time(example):
    # Parse date and time strings
    date = datetime.strptime(date_str, "%Y-%m-%d")
    time = datetime.strptime(time_str, "%H:%M:%S").time()
    
    # Check if time is past 17:00
    if perigon_apple.pubTime() > perigon_apple.pubTime("17:00:00", "%H:%M:%S").time():
        # Increment date by one day
        adjusted_date = perigon_apple.pubDate_Clean + timedelta(days=1)
        adjusted_date_str = adjusted_date.strftime("%Y-%m-%d")
        return adjusted_date_str
    else:
        return date_str
    
#where are we running this?

In [43]:
perigon_apple['pubDate'] = perigon_apple['pubDate_Clean'].apply(adjust_date_time)

NameError: name 'datetime' is not defined

In [44]:
#make a title sentiment analyzer using textblob
#polarity ranges from -1 to 1, 0 is no polarity
from textblob import TextBlob

#Create a function to get the polarity
def title_polarity(example):
    return TextBlob(example).sentiment.polarity
  
 #Create two new columns ‘Subjectivity’ & ‘Polarity’
perigon_apple['title_polarity'] = perigon_apple['title'].apply(title_polarity)


In [45]:
from textblob import TextBlob

#Create a function to get the polarity
def title_subjectivity(example):
    return TextBlob(example).sentiment.subjectivity
  
 #Create two new columns ‘Subjectivity'
perigon_apple['title_subjectivity'] = perigon_apple['title'].apply(title_subjectivity)

In [46]:
#here's how to explode the data
perigon_apple = perigon_apple[['summary','title','pubDate','addDate','n_companies','StockSymbol','authorsByline','source','companies','entities','positive_sentiment', 'negative_sentiment', 'neutral_sentiment']]

#EXPLODE, make a new row for each company.

perigon_apple = perigon_apple.explode('companies')
# Reset index to maintain the original index values
perigon_apple.reset_index(drop=True, inplace=True)

#df_exploded.shape

perigon_apple.pubDate = pd.to_datetime(perigon_apple.pubDate)
perigon_apple.addDate = pd.to_datetime(perigon_apple.addDate)





KeyError: "['n_companies', 'positive_sentiment', 'negative_sentiment', 'neutral_sentiment'] not in index"

In [ ]:
#create a subset of exploded data that is only the established ticker


#question about calling articles:
#is it possible to collect the same article twice when searching by stock ticker for two different companies that 
#are both featured in the same article?

In [ ]:
yahoo_apple_ subset = yahoo[yahoo['StockSymbol'] is in ['companies']]

In [16]:
#do this after we convert daily data into weekly data
#this merges the two datasets
for k, k_row in perigon_apple.iterrows():
    for i, i_row in weekly_yahoo_apple.iterrows():
        if weekly_yahoo_apple.loc[i, 'start_date'] <= perigon_apple.loc[k, 'pubDate'] <= weekly_yahoo_apple.loc[i, 'end_date']:
            perigon_apple.at[k, 'open'] = weekly_yahoo_apple.at[i, 'open']
            perigon_apple.at[k, 'close'] = weekly_yahoo_apple.at[i, 'close']
            perigon_apple.at[k, 'ticker'] = weekly_yahoo_apple.at[i, 'ticker']
            perigon_apple['high'] = weekly_yahoo_apple['high']
            perigon_apple['low'] = weekly_yahoo_apple['low']
            perigon_apple['adjclose'] = weekly_yahoo_apple['adjclose']
            perigon_apple['volume'] = weekly_yahoo_apple['volume']
            break  


SyntaxError: invalid syntax (3035703265.py, line 9)